This notebook is intended for editing the connector records without creating project cards. The reasons of not doing this in project cards are: 1) Applying them in project cards requires having centroids in the network before the project cards are applied. metcouncil.add_centroid_and_centroid_connector() step needs to be called before the model network is created, and the centroids will be added into the standard network. We will need to revise the notebook, and if necessary, related Lasso codes. 2) it keeps centroids separate from the standard network, no confusion, and 3) if in the future we want to rebuild centroids, it will not touch standard network.

In [1]:
import geopandas as gpd
import pandas as pd
import os
import hashlib
from shapely.geometry import Point, LineString

In [2]:
# user to define the paths to the data directory
centroid_data_dir = r"D:\metcouncil_network_rebuild\data\processed\version_00\standard_networks"

input_dir = r"D:\metcouncil_network_rebuild\data\processed\version_00\standard_networks"

output_dir = r"D:\metcouncil_network_rebuild\data\processed\version_00\standard_networks\new_centroids"

In [16]:
taz_node_gdf = pd.read_pickle(os.path.join(centroid_data_dir, "centroid_node.pickle"))
taz_cc_link_gdf = pd.read_pickle(os.path.join(centroid_data_dir, "cc_link.pickle"))
taz_cc_shape_gdf = pd.read_pickle(os.path.join(centroid_data_dir, "cc_shape.pickle"))

In [17]:
print(taz_node_gdf.shape)
print(taz_cc_link_gdf.shape)
print(taz_cc_shape_gdf.shape)

(3030, 8)
(41812, 14)
(20904, 4)


## Example of deleting centroid connectors and associated records

In [19]:
# deleting the connectors based on A Node, B Node. 
# The user to supply the A/B nodes of targeted connectors. links are uniquely identified by A/B nodes tuple such as (1,6452)

a_b_tuples_list = [(1874, 125480), (125480, 1874)]

# to define more than one link records, and more tuples to the list.
# a_b_tuples_list = [(1874, 125480)]
# a_b_tuples_list = [(1874, 125480), (1874, 125481), (1874, 125481)]

# to delete the links based on the A/B nodes tuple list
taz_cc_link_gdf = taz_cc_link_gdf[~taz_cc_link_gdf.set_index(['A', 'B']).index.isin(a_b_tuples_list)].reset_index(drop=True)

# update the shapes records based on the updated links
taz_cc_shape_gdf = taz_cc_shape_gdf[taz_cc_shape_gdf.id.isin(taz_cc_link_gdf.shstGeometryId)]

In [20]:
print(taz_node_gdf.shape)
print(taz_cc_link_gdf.shape)
print(taz_cc_shape_gdf.shape)

(3030, 8)
(41810, 14)
(20903, 4)


In [21]:
taz_cc_shape_gdf[taz_cc_shape_gdf.id == 'e1c3ebad0eca043c4ddff68a64b782da']

,id,fromIntersectionId,county,geometry


## Example of adding centroid connectors and associated records

In [22]:
# requires the standard network node data
node_file = os.path.join(input_dir, 'nodes.geojson')

nodes_gdf = gpd.read_file(node_file)

In [23]:
# how the links data looks like
taz_cc_link_gdf.iloc[0]

A                                                                   1.0
B                                                               38291.0
drive_access                                                          1
walk_access                                                           0
bike_access                                                           0
shstGeometryId                         5f5213170c569d0ac15786e7e6fbc43b
id                                     5f5213170c569d0ac15786e7e6fbc43b
u                                                                   NaN
v                                                           186172989.0
fromIntersectionId                                                 None
toIntersectionId                       1d37520ff9747f51e7a9754693a19a8d
county                                                            Anoka
roadway                                                             taz
geometry              LINESTRING (-93.47923800000001 45.406446, 

In [24]:
# Adding the connectors based on A Node, B Node. 
# The user to supply the A/B nodes of targeted connectors. links are uniquely identified by A/B nodes tuple such as (1,6452)

# input fields that need users to define values
# A, B, drive_access, walk_access, bike_access, county, roadway
# the rest of the field can be automatically populated

# for example, to add a link between A node 1874 and B node 125482:
add_link_dict = {
    "A": [1874],
    "B": [125482],
    "drive_access": [1],
    "walk_access": [1],
    "bike_access": [1],
    "county": ['Ramsey'],
    "roadway": ['taz'],
}

# to add more than one records
# add_link_dict = {
#     "A": [1874, 1874],
#     "B": [125482, 125483],
#     "drive_access": [1, 1],
#     "walk_access": [1, 1],
#     "bike_access": [1, 1],
#     "county": ['Ramsey', 'Ramsey'],
#     "roadway": ['taz', 'taz'],
# }

# new centroid links
new_centroid_links_df = pd.DataFrame(add_link_dict)

In [26]:
# link geometries are created based on nodes
# create a node data base that includes both street nodes and centroids

all_nodes_gdf = pd.concat([nodes_gdf, taz_node_gdf], ignore_index=True)
all_nodes_gdf = all_nodes_gdf[["model_node_id", "osm_node_id", "shst_node_id", "geometry"]].drop_duplicates()

osm_dict = dict(zip(nodes_gdf.model_node_id, nodes_gdf.osm_node_id))
shst_dict = dict(zip(all_nodes_gdf.model_node_id, all_nodes_gdf.shst_node_id))

In [27]:
# map u,v,fromIntersectionId,toIntersectionId

new_centroid_links_df['u'] = new_centroid_links_df.A.map(osm_dict)
new_centroid_links_df['v'] = new_centroid_links_df.B.map(osm_dict)
new_centroid_links_df['fromIntersectionId'] = new_centroid_links_df.A.map(shst_dict)
new_centroid_links_df['toIntersectionId'] = new_centroid_links_df.B.map(shst_dict)

In [28]:
# check how shapes data look like
taz_cc_shape_gdf.iloc[0]

id                                     0000f69d16eddb948334f8e35764f164
fromIntersectionId                     9a855a66aa837802031a97e161d42831
county                                                         Hennepin
geometry              LINESTRING (-93.42102840000001 44.908365, -93....
Name: 0, dtype: object

In [29]:
# create geometry based on A node B node coordinates
new_centroid_links_df['a_node_geometry'] = new_centroid_links_df.A.map(all_nodes_gdf.set_index('model_node_id').geometry)
new_centroid_links_df['b_node_geometry'] = new_centroid_links_df.B.map(all_nodes_gdf.set_index('model_node_id').geometry)

# create link geometry based on A node B node geometires
new_centroid_links_df['geometry'] = new_centroid_links_df.apply(lambda x: LineString([x.a_node_geometry, x.b_node_geometry]), axis=1)

c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_wsp\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [30]:
def create_unique_shape_id(line_string: LineString):
    """
    Creates a unique hash id using the coordinates of the geomtery

    Args:
    line_string: Line Geometry as a LineString

    Returns: string
    """

    x1, y1 = list(line_string.coords)[0]  # first co-ordinate (A node)
    x2, y2 = list(line_string.coords)[-1]  # last co-ordinate (B node)

    message = "Geometry {} {} {} {}".format(x1, y1, x2, y2)
    unhashed = message.encode("utf-8")
    hash = hashlib.md5(unhashed).hexdigest()

    return hash

In [31]:
# create unique shape id for the new links
new_centroid_links_df["shstGeometryId"] = new_centroid_links_df["geometry"].apply(
        lambda x: create_unique_shape_id(x)
    )
new_centroid_links_df["id"] = new_centroid_links_df["shstGeometryId"]

In [32]:
# get unique new shape records

new_centroid_shapes_df = new_centroid_links_df[["id", "geometry", "county", "fromIntersectionId"]].drop_duplicates(subset=['id'])

# check they do not overlap with existing shapes

assert len(new_centroid_shapes_df[new_centroid_shapes_df.id.isin(taz_cc_shape_gdf.id)]) == 0

In [33]:
# append the new links and shapes to the existing links and shapes

taz_cc_link_gdf = pd.concat([taz_cc_link_gdf, new_centroid_links_df[[c for c in new_centroid_links_df.columns if c in taz_cc_link_gdf.columns]]], ignore_index=True, sort=False)
taz_cc_shape_gdf = pd.concat([taz_cc_shape_gdf, new_centroid_shapes_df[[c for c in new_centroid_shapes_df.columns if c in taz_cc_shape_gdf.columns]]], ignore_index=True, sort=False)

In [34]:
print(taz_node_gdf.shape)
print(taz_cc_link_gdf.shape)
print(taz_cc_shape_gdf.shape)

(3030, 8)
(41811, 14)
(20904, 4)


## Write out

In [36]:
# write out back to pickle

taz_node_gdf.to_pickle(os.path.join(output_dir, "centroid_node.pickle"))
taz_cc_link_gdf.to_pickle(os.path.join(output_dir, "cc_link.pickle"))
taz_cc_shape_gdf.to_pickle(os.path.join(output_dir, "cc_shape.pickle"))

In [37]:
# write nodes and shapes to geojson, write links to json

taz_node_gdf.to_file(os.path.join(output_dir, "centroid_node.geojson"), driver="GeoJSON")
with open(os.path.join(output_dir, "cc_link.json"), 'w') as f:
    f.write(pd.DataFrame(taz_cc_link_gdf.drop(columns=['geometry'])).to_json(orient='records'))
taz_cc_link_gdf.to_file(os.path.join(output_dir, "cc_link.geojson"), driver="GeoJSON")
taz_cc_shape_gdf.to_file(os.path.join(output_dir, "cc_shape.geojson"), driver="GeoJSON")